In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
import pdb
import _pickle as cPickle
import time
import subprocess
import glob

import random
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import defaultdict, Counter
from ncrp import Topic, Doc, init, sample, get_perplexity, get_topic_specialization, get_hierarchical_affinities, get_freq_tokens_ncrp, get_docs
from configure import get_config

# load config & data 

In [3]:
config = get_config(nb_name)
np.random.seed(config.seed)
random.seed(config.seed)

In [4]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))

Exception ignored in: <_io.FileIO name='data/bags/instances.pkl' mode='rb' closefd=True>


In [5]:
config.n_doc = len(instances_train)
config.n_vocab = len(bow_idxs)
config.n_doc, config.n_vocab

(31943, 1035)

In [6]:
docs_bow = [instance.bow for instance in instances_train]
docs_raw = [[[bow_index]*int(doc_bow[bow_index]) for bow_index in np.where(doc_bow > 0)[0]] for doc_bow in docs_bow]
docs_words = [[idx for idxs in doc for idx in idxs] for doc in docs_raw]
np.sum([len(doc_words) for doc_words in docs_words])

568401

# run

## initialize log

In [7]:
checkpoint = []
ppl_min = np.inf
epoch = 0

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','VALID:','TEST:','SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','PPL','PPL', 'PPL','1', '2', '3', 'CHILD', 'OTHER']]))))

def update_checkpoint(config, checkpoint, epoch):
    checkpoint.append(config.path_model + '-%i' % epoch)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0)
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize data

In [8]:
topic_root = Topic(idx='0', sibling_idx=0, parent=None, depth=0, config=config)
train_docs = get_docs(instances_train, config)
dev_docs = get_docs(instances_dev, config)
test_docs = get_docs(instances_test, config)
init(train_docs, dev_docs, test_docs, topic_root)

0 10000 20000 30000 0 0 

## run

In [9]:
time_start = time.time()
while epoch < config.n_epochs:
    sample(train_docs, dev_docs, test_docs, topic_root)
    ppl_train = get_perplexity(train_docs, topic_root)
    ppl_dev = get_perplexity(dev_docs, topic_root)
    
    if ppl_dev < ppl_min:
        ppl_min = ppl_dev
        ppl_test = get_perplexity(test_docs, topic_root)
        cPickle.dump([test_docs, topic_root], open(config.path_model + '-%i'%epoch, 'wb'))
        update_checkpoint(config, checkpoint, epoch)
        
    depth_spec = get_topic_specialization(test_docs, topic_root)
    hierarchical_affinities = get_hierarchical_affinities(topic_root)
    
    clear_output()
    time_log = int(time.time() - time_start)
    time_start = time.time()
    log_series = pd.Series([time_log, epoch, 0, \
            '%.0f'%ppl_train, '%.0f'%ppl_dev, '%.0f'%ppl_test, \
            '%.2f'%depth_spec[1], '%.2f'%depth_spec[2], '%.2f'%depth_spec[3], \
            '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
            index=log_df.columns)
    log_df.loc[epoch] = log_series
    display(log_df)
    get_freq_tokens_ncrp(topic_root, idx_to_word, bow_idxs)
    
    cPickle.dump(log_df, open(config.path_log, 'wb'))
    epoch += 1

TRAIN: VALID: TEST: SPEC:             HIER:      
   Time  Ep Ct    PPL    PPL   PPL     1     2     3 CHILD OTHER
0   118   0  0    539    513   509  0.06  0.30  0.47  0.80  0.73
1   123   1  0    530    508   502  0.06  0.32  0.48  0.77  0.70
2   123   2  0    514    498   489  0.07  0.36  0.49  0.71  0.61
3   121   3  0    481    465   456  0.09  0.31  0.50  0.67  0.51
4   134   4  0    457    445   439  0.09  0.32  0.50  0.75  0.61
5   137   5  0    452    444   438  0.10  0.31  0.51  0.71  0.59
6   149   6  0    449    441   435  0.10  0.29  0.52  0.65  0.56
7   149   7  0    440    432   426  0.10  0.28  0.52  0.59  0.48
8   149   8  0    431    427   419  0.11  0.27  0.52  0.52  0.44
9   163   9  0    426    421   415  0.11  0.33  0.52  0.63  0.54
10  163  10  0    422    415   411  0.11  0.34  0.53  0.61  0.52
11  162  11  0    418    412   407  0.11  0.34  0.53  0.59  0.49
12  161  12  0    413    407   402  0.11  0.34  0.53  0.57  0.46
13  161  13  0    406    400   396  0.11  0.33  0.53  0.55  0.44
14  160  14  0    401    396   390  0.11  0.31  0.54  0.53  0.42
15  165  15  0    397    392   386  0.11  0.31  0.54  0.51  0.41
16  168  16  0    394    389   384  0.11  0.31  0.54  0.49  0.40
17  184  17  0    390    386   381  0.11  0.36  0.54  0.58  0.48
18  188  18  0    387    383   377  0.11  0.37  0.54  0.57  0.47
19  189  19  0    383    379   376  0.11  0.37  0.55  0.55  0.45
20  188  20  0    381    376   373  0.11  0.38  0.55  0.54  0.44
21  187  21  0    380    377   373  0.11  0.37  0.55  0.52  0.43
22  194  22  0    378    375   368  0.11  0.36  0.54  0.50  0.42
23  190  23  0    373    372   364  0.11  0.36  0.55  0.48  0.38
24  203  24  0    369    369   359  0.11  0.40  0.56  0.56  0.45
25  207  25  0    367    366   356  0.11  0.40  0.55  0.56  0.45
26  207  26  0    365    364   355  0.11  0.41  0.56  0.54  0.44
27  207  27  0    364    364   354  0.11  0.41  0.56  0.52  0.43
28  208  28  0    363    362   355  0.11  0.41  0.56  0.51  0.42
29  207  29  0    361    360   355  0.11  0.42  0.56  0.49  0.40

 0 31943 236572.0 ! ; & nice price quality bought - inside perfect
   0-1 3563 34282.0 ! love color cover price recommend perfect buy ... ordered
     0-1-1 3563 1524.0 cards memory arrived sd easy highly months black card pink
   0-2 9873 89119.0 carry pockets room ! plenty back compartment pocket shoulder strap
     0-2-1 9844 7884.0 cards camera small memory canon lenses lens sd top space
     0-2-2 29 9.0 waterproof recommend snaps portable roomy caught carry-on found fine finger
   0-3 6338 69343.0 cover ! color keyboard love bottom mac pro hard easy
     0-3-1 5115 10211.0 bottom top feet part plastic months corners speck piece cracked
     0-3-2 1223 502.0 dont brown reasonable clean items ordering hands leaves middle material
   0-4 6985 51662.0 sleeve pro air smell cover zipper protection protect inch hard
     0-4-1 1501 2329.0 bottom plastic corners rubber review top months week corner speck
     0-4-2 5484 5874.0 foam protection memory sleeve neoprene netbook zipper cards b

Exception ignored in: <_io.FileIO name='model/bags/ncrp/bags-mncrp-alp10.50.1-eta5-gam0.01-epoch30/log' mode='wb' closefd=True>
